In [6]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
# IMPORT PY-FESOM MODULES
sys.path.append("codes/modules")
sys.path.append("codes/pyfesom-master/") # add pyfesom to search path
#########################################

computer =  blogin1
working directory set to /home/hbkoziel/awi-models


In [7]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
#import colormaps as cmaps
from matplotlib.colors import ListedColormap
import pyfesom as pf

In [18]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = plt.get_cmap('RdBu_r')
# figure export 
export_plot = True
# figure export definition
dpicnt=150
# choose simulation
runid='Arc12'
# choose date
first_year = 1980
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(4,8,5).astype(int)
# choose depth
get3d = False
# load the given biological tracer #
phyc_id, phychl_id, diac_id, diachl_id, par_id, temp_id, depth	='tr05', 'tr06', 'tr14', 'tr15', 'PAR3D_mean', 'temp', 0
phyn_id, dian_id, diasi_id, din_id, dsi_id, dfe_id, ice_id	='tr04', 'tr13', 'tr16', 'tr01', 'tr18', 'tr19', 'area'
# export CSV
export_csv = True
# model parameters
alfa_phy, alfa_dia, Pcm, C2K, rTref, Ae = 0.19, 0.26, 3.0, 273.15, 1/288.15, 4500.0
NMinSlope, SiMinSlope, NCmin, SiCmin  = 50, 1000, 0.04, 0.04

########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'
outputpath = '/scratch/usr/hbkoziel/Arc12/CSV/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /home/hbkoziel/awi-models/figures/Arc12/ already existing


In [9]:
mesh = fesom_mesh(meshpath, get3d = get3d)

Load 2d part of the grid in 3 second(s)


In [24]:
alfa_phy, alfa_dia, Pcm, C2K, rTref, Ae = 0.19, 0.26, 3.0, 273.15, 1/288.15, 4500.0
NMinSlope, SiMinSlope, NCmin, SiCmin  = 50, 1000, 0.04, 0.04

phyc_id, phychl_id, diac_id, diachl_id, par_id, temp_id, depth	='tr05', 'tr06', 'tr14', 'tr15', 'PAR3D_mean', 'temp', 0
phyn_id, dian_id, diasi_id, din_id, dsi_id, dfe_id, ice_id	='tr04', 'tr13', 'tr16', 'tr01', 'tr18', 'tr19', 'area'

limdia1 = np.zeros(shape=(len(years),len(mesh.x2)))
limphy1 = np.zeros(shape=(len(years),len(mesh.x2)))

for ind in range(0,len(years)):
    if years[ind] >= 2000:
        Felimphy          = np.zeros(shape=(76,len(mesh.x2)))
        Felimdia          = np.zeros(shape=(76,len(mesh.x2)))
        Nlimphy           = np.zeros(shape=(76,len(mesh.x2)))
        Nlimdia           = np.zeros(shape=(76,len(mesh.x2)))
        Silim             = np.zeros(shape=(76,len(mesh.x2)))
        Phy_Light_limiter = np.zeros(shape=(76,len(mesh.x2)))
        Dia_Light_limiter = np.zeros(shape=(76,len(mesh.x2)))
        print 'Entered year loop',years[ind]
        ncfile      = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'+runid+'.'+str(years[ind])+'.oce.mean.nc'
        f           = Dataset(ncfile, 'r')

        DIN3D       = f.variables[din_id][60:136,0:mesh.n2d]
        DSi3D       = f.variables[dsi_id][60:136,0:mesh.n2d]
        DFe3D       = f.variables[dfe_id][60:136,0:mesh.n2d]
        phyc3D      = f.variables[phyc_id][60:136,0:mesh.n2d]
        phyn3D      = f.variables[phyn_id][60:136,0:mesh.n2d]
        phychl3D    = f.variables[phychl_id][60:136,0:mesh.n2d]
        diac3D      = f.variables[diac_id][60:136,0:mesh.n2d]
        dian3D      = f.variables[dian_id][60:136,0:mesh.n2d]
        diasi3D     = f.variables[diasi_id][60:136,0:mesh.n2d]
        diachl3D    = f.variables[diachl_id][60:136,0:mesh.n2d]
        T3D         = f.variables[temp_id][60:136,0:mesh.n2d]

        ncfile      = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'+runid+'.'+str(years[ind])+'.bio.mean.nc'
        f           = Dataset(ncfile, 'r')
        par3D       = f.variables[par_id][60:136,0:mesh.n2d]

        m=0 
        for cnt in range(0, 76): # For every month in every year, the limitation is calculated
            print 'year: ',years[ind],', day: ',cnt+1
            DIN2D         = DIN3D[m,:]
            Nlimphy[m,:]  = Nlimphy[m,:] + DIN2D/(DIN2D+0.55)
            Nlimdia[m,:]  = Nlimdia[m,:] + DIN2D/(DIN2D+1.0)

            DSi2D         = DSi3D[m,:]
            Silim[m,:]    = Silim[m,:] + DSi2D/(DSi2D+4.0)

            DFe2D         = DFe3D[m,:]
            Felimphy[m,:] = Felimphy[m,:] + DFe2D/(DFe2D+0.02)
            Felimdia[m,:] = Felimdia[m,:] + DFe2D/(DFe2D+0.12)

            # The correct depth for 2d field 
            phyc2D   = phyc3D[m,:]
            phyn2D   = phyn3D[m,:]
            phychl2D = phychl3D[m,:]
            diac2D   = diac3D[m,:]
            dian2D   = dian3D[m,:]
            diasi2D  = diasi3D[m,:]
            diachl2D = diachl3D[m,:]
            par2d    = par3D[m,:]
            T2d      = T3D[m,:]

            # Quotas
            phychl2c = phychl2D/phyc2D 
            phyn2c   = phyn2D/phyc2D
            diachl2c = diachl2D/diac2D
            dian2c   = dian2D/diac2D
            diasi2c  = diasi2D/diac2D

            # Temperature dependece
            T2d = 1./(T2d + C2K)
            T2d = np.exp( -Ae * (T2d - rTref));

            # Nutrient growth limitation
            Phy_dq              = NCmin - phyn2c
            Phy_recom_limiter   = 1.0 - np.exp( -NMinSlope*( abs(Phy_dq)-Phy_dq )**2)

            Dia_dq              = NCmin - dian2c
            Dia_recom_limiterN  = 1.0 - np.exp( -NMinSlope*( abs(Dia_dq)-Dia_dq )**2)
            Dia_dq              = SiCmin - diasi2c;
            Dia_recom_limiterSi = 1.0 - np.exp( -SiMinSlope*( abs(Dia_dq)-Dia_dq )**2)

            # Most limiting factor 
            Fephy       = DFe2D/(DFe2D+0.02)
            Phy_qlimFac = np.column_stack((Phy_recom_limiter,Fephy))
            Phy_qlimFac = Phy_qlimFac.min(axis=1)

            Fedia       = DFe2D/(DFe2D+0.12)
            Dia_qlimFac = np.column_stack((Dia_recom_limiterN,Dia_recom_limiterSi,Fedia))
            Dia_qlimFac = Dia_qlimFac.min(axis=1)

            # pmax
            Pmax_phy = Pcm * Phy_qlimFac * T2d
            Pmax_dia = Pcm * Dia_qlimFac * T2d

            Phy_Light_limiter[m,:] = Phy_Light_limiter[m,:] + 1 - np.exp((-alfa_phy * phychl2c * par2d)/Pmax_phy)
            Dia_Light_limiter[m,:] = Dia_Light_limiter[m,:] + 1 - np.exp((-alfa_dia * diachl2c * par2d)/Pmax_dia)
            m=m+1
    else:
        Felimphy          = np.zeros(shape=(5,len(mesh.x2)))
        Felimdia          = np.zeros(shape=(5,len(mesh.x2)))
        Nlimphy           = np.zeros(shape=(5,len(mesh.x2)))
        Nlimdia           = np.zeros(shape=(5,len(mesh.x2)))
        Silim             = np.zeros(shape=(5,len(mesh.x2)))
        Phy_Light_limiter = np.zeros(shape=(5,len(mesh.x2)))
        Dia_Light_limiter = np.zeros(shape=(5,len(mesh.x2)))
        print 'Entered year loop',years[ind]
        ncfile      = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'+runid+'.'+str(years[ind])+'.oce.mean.nc'
        f           = Dataset(ncfile, 'r')

        DIN3D       = f.variables[din_id][4:9,0:mesh.n2d]
        DSi3D       = f.variables[dsi_id][4:9,0:mesh.n2d]
        DFe3D       = f.variables[dfe_id][4:9,0:mesh.n2d]
        phyc3D      = f.variables[phyc_id][4:9,0:mesh.n2d]
        phyn3D      = f.variables[phyn_id][4:9,0:mesh.n2d]
        phychl3D    = f.variables[phychl_id][4:9,0:mesh.n2d]
        diac3D      = f.variables[diac_id][4:9,0:mesh.n2d]
        dian3D      = f.variables[dian_id][4:9,0:mesh.n2d]
        diasi3D     = f.variables[diasi_id][4:9,0:mesh.n2d]
        diachl3D    = f.variables[diachl_id][4:9,0:mesh.n2d]
        T3D         = f.variables[temp_id][4:9,0:mesh.n2d]

        ncfile      = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'+runid+'.'+str(years[ind])+'.bio.mean.nc'
        f           = Dataset(ncfile, 'r')
        par3D       = f.variables[par_id][4:9,0:mesh.n2d]

        m=0 
        for cnt in range(0, 5): # For every month in every year, the limitation is calculated
            print 'year: ',years[ind],', month: ',cnt+1
            DIN2D         = DIN3D[m,:]
            Nlimphy[m,:]  = Nlimphy[m,:] + DIN2D/(DIN2D+0.55)
            Nlimdia[m,:]  = Nlimdia[m,:] + DIN2D/(DIN2D+1.0)

            DSi2D         = DSi3D[m,:]
            Silim[m,:]    = Silim[m,:] + DSi2D/(DSi2D+4.0)

            DFe2D         = DFe3D[m,:]
            Felimphy[m,:] = Felimphy[m,:] + DFe2D/(DFe2D+0.02)
            Felimdia[m,:] = Felimdia[m,:] + DFe2D/(DFe2D+0.12)

            # The correct depth for 2d field 
            phyc2D   = phyc3D[m,:]
            phyn2D   = phyn3D[m,:]
            phychl2D = phychl3D[m,:]
            diac2D   = diac3D[m,:]
            dian2D   = dian3D[m,:]
            diasi2D  = diasi3D[m,:]
            diachl2D = diachl3D[m,:]
            par2d    = par3D[m,:]
            T2d      = T3D[m,:]

            # Quotas
            phychl2c = phychl2D/phyc2D 
            phyn2c   = phyn2D/phyc2D
            diachl2c = diachl2D/diac2D
            dian2c   = dian2D/diac2D
            diasi2c  = diasi2D/diac2D

            # Temperature dependece
            T2d = 1./(T2d + C2K)
            T2d = np.exp( -Ae * (T2d - rTref));

            # Nutrient growth limitation
            Phy_dq              = NCmin - phyn2c
            Phy_recom_limiter   = 1.0 - np.exp( -NMinSlope*( abs(Phy_dq)-Phy_dq )**2)

            Dia_dq              = NCmin - dian2c
            Dia_recom_limiterN  = 1.0 - np.exp( -NMinSlope*( abs(Dia_dq)-Dia_dq )**2)
            Dia_dq              = SiCmin - diasi2c;
            Dia_recom_limiterSi = 1.0 - np.exp( -SiMinSlope*( abs(Dia_dq)-Dia_dq )**2)

            # Most limiting factor 
            Fephy       = DFe2D/(DFe2D+0.02)
            Phy_qlimFac = np.column_stack((Phy_recom_limiter,Fephy))
            Phy_qlimFac = Phy_qlimFac.min(axis=1)

            Fedia       = DFe2D/(DFe2D+0.12)
            Dia_qlimFac = np.column_stack((Dia_recom_limiterN,Dia_recom_limiterSi,Fedia))
            Dia_qlimFac = Dia_qlimFac.min(axis=1)

            # pmax
            Pmax_phy = Pcm * Phy_qlimFac * T2d
            Pmax_dia = Pcm * Dia_qlimFac * T2d

            Phy_Light_limiter[m,:] = Phy_Light_limiter[m,:] + 1 - np.exp((-alfa_phy * phychl2c * par2d)/Pmax_phy)
            Dia_Light_limiter[m,:] = Dia_Light_limiter[m,:] + 1 - np.exp((-alfa_dia * diachl2c * par2d)/Pmax_dia)
            m=m+1
    Felimphy          = np.mean(Felimphy, axis=0)
    Felimdia          = np.mean(Felimdia, axis=0)
    Nlimdia           = np.mean(Nlimdia, axis=0)
    Nlimphy           = np.mean(Nlimphy, axis=0)
    Silim             = np.mean(Silim, axis=0)
    Phy_Light_limiter = np.mean(Phy_Light_limiter, axis=0)       
    Dia_Light_limiter = np.mean(Dia_Light_limiter, axis=0) #[n2d months] => [n2d]
    lim=np.column_stack((Felimdia,Nlimdia,Silim,Dia_Light_limiter))
    limdia1[ind,:]=lim.argmin(axis=1)
    lim=np.column_stack((Felimphy,Nlimphy,Phy_Light_limiter))
    limphy1[ind,:]=lim.argmin(axis=1)

Entered year loop 1980
year:  1980 , month:  1
year:  1980 , month:  2


/home/hbkoziel/.conda/envs/pyfesom-docs/lib/python2.7/site-packages/ipykernel_launcher.py:187: RuntimeWarning: divide by zero encountered in divide


year:  1980 , month:  3
year:  1980 , month:  4


/home/hbkoziel/.conda/envs/pyfesom-docs/lib/python2.7/site-packages/ipykernel_launcher.py:186: RuntimeWarning: divide by zero encountered in divide


year:  1980 , month:  5
Entered year loop 1981
year:  1981 , month:  1
year:  1981 , month:  2
year:  1981 , month:  3
year:  1981 , month:  4
year:  1981 , month:  5
Entered year loop 1982
year:  1982 , month:  1
year:  1982 , month:  2
year:  1982 , month:  3
year:  1982 , month:  4
year:  1982 , month:  5
Entered year loop 1983
year:  1983 , month:  1
year:  1983 , month:  2
year:  1983 , month:  3
year:  1983 , month:  4
year:  1983 , month:  5
Entered year loop 1984
year:  1984 , month:  1
year:  1984 , month:  2
year:  1984 , month:  3
year:  1984 , month:  4
year:  1984 , month:  5
Entered year loop 1985
year:  1985 , month:  1
year:  1985 , month:  2
year:  1985 , month:  3
year:  1985 , month:  4
year:  1985 , month:  5
Entered year loop 1986
year:  1986 , month:  1
year:  1986 , month:  2
year:  1986 , month:  3
year:  1986 , month:  4
year:  1986 , month:  5
Entered year loop 1987
year:  1987 , month:  1
year:  1987 , month:  2
year:  1987 , month:  3
year:  1987 , month:  4

/home/hbkoziel/.conda/envs/pyfesom-docs/lib/python2.7/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in divide
/home/hbkoziel/.conda/envs/pyfesom-docs/lib/python2.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: divide by zero encountered in divide


year:  2000 , day:  2
year:  2000 , day:  3
year:  2000 , day:  4
year:  2000 , day:  5
year:  2000 , day:  6
year:  2000 , day:  7
year:  2000 , day:  8
year:  2000 , day:  9
year:  2000 , day:  10
year:  2000 , day:  11
year:  2000 , day:  12
year:  2000 , day:  13
year:  2000 , day:  14
year:  2000 , day:  15
year:  2000 , day:  16
year:  2000 , day:  17
year:  2000 , day:  18
year:  2000 , day:  19
year:  2000 , day:  20
year:  2000 , day:  21
year:  2000 , day:  22
year:  2000 , day:  23
year:  2000 , day:  24
year:  2000 , day:  25
year:  2000 , day:  26
year:  2000 , day:  27
year:  2000 , day:  28
year:  2000 , day:  29
year:  2000 , day:  30
year:  2000 , day:  31
year:  2000 , day:  32
year:  2000 , day:  33
year:  2000 , day:  34
year:  2000 , day:  35
year:  2000 , day:  36
year:  2000 , day:  37
year:  2000 , day:  38
year:  2000 , day:  39
year:  2000 , day:  40
year:  2000 , day:  41
year:  2000 , day:  42
year:  2000 , day:  43
year:  2000 , day:  44
year:  2000 , day: 

year:  2004 , day:  54
year:  2004 , day:  55
year:  2004 , day:  56
year:  2004 , day:  57
year:  2004 , day:  58
year:  2004 , day:  59
year:  2004 , day:  60
year:  2004 , day:  61
year:  2004 , day:  62
year:  2004 , day:  63
year:  2004 , day:  64
year:  2004 , day:  65
year:  2004 , day:  66
year:  2004 , day:  67
year:  2004 , day:  68
year:  2004 , day:  69
year:  2004 , day:  70
year:  2004 , day:  71
year:  2004 , day:  72
year:  2004 , day:  73
year:  2004 , day:  74
year:  2004 , day:  75
year:  2004 , day:  76
Entered year loop 2005
year:  2005 , day:  1
year:  2005 , day:  2
year:  2005 , day:  3
year:  2005 , day:  4
year:  2005 , day:  5
year:  2005 , day:  6
year:  2005 , day:  7
year:  2005 , day:  8
year:  2005 , day:  9
year:  2005 , day:  10
year:  2005 , day:  11
year:  2005 , day:  12
year:  2005 , day:  13
year:  2005 , day:  14
year:  2005 , day:  15
year:  2005 , day:  16
year:  2005 , day:  17
year:  2005 , day:  18
year:  2005 , day:  19
year:  2005 , day:  

year:  2009 , day:  28
year:  2009 , day:  29
year:  2009 , day:  30
year:  2009 , day:  31
year:  2009 , day:  32
year:  2009 , day:  33
year:  2009 , day:  34
year:  2009 , day:  35
year:  2009 , day:  36
year:  2009 , day:  37
year:  2009 , day:  38
year:  2009 , day:  39
year:  2009 , day:  40
year:  2009 , day:  41
year:  2009 , day:  42
year:  2009 , day:  43
year:  2009 , day:  44
year:  2009 , day:  45
year:  2009 , day:  46
year:  2009 , day:  47
year:  2009 , day:  48
year:  2009 , day:  49
year:  2009 , day:  50
year:  2009 , day:  51
year:  2009 , day:  52
year:  2009 , day:  53
year:  2009 , day:  54
year:  2009 , day:  55
year:  2009 , day:  56
year:  2009 , day:  57
year:  2009 , day:  58
year:  2009 , day:  59
year:  2009 , day:  60
year:  2009 , day:  61
year:  2009 , day:  62
year:  2009 , day:  63
year:  2009 , day:  64
year:  2009 , day:  65
year:  2009 , day:  66
year:  2009 , day:  67
year:  2009 , day:  68
year:  2009 , day:  69
year:  2009 , day:  70
year:  2009

year:  2014 , day:  1
year:  2014 , day:  2
year:  2014 , day:  3
year:  2014 , day:  4
year:  2014 , day:  5
year:  2014 , day:  6
year:  2014 , day:  7
year:  2014 , day:  8
year:  2014 , day:  9
year:  2014 , day:  10
year:  2014 , day:  11
year:  2014 , day:  12
year:  2014 , day:  13
year:  2014 , day:  14
year:  2014 , day:  15
year:  2014 , day:  16
year:  2014 , day:  17
year:  2014 , day:  18
year:  2014 , day:  19
year:  2014 , day:  20
year:  2014 , day:  21
year:  2014 , day:  22
year:  2014 , day:  23
year:  2014 , day:  24
year:  2014 , day:  25
year:  2014 , day:  26
year:  2014 , day:  27
year:  2014 , day:  28
year:  2014 , day:  29
year:  2014 , day:  30
year:  2014 , day:  31
year:  2014 , day:  32
year:  2014 , day:  33
year:  2014 , day:  34
year:  2014 , day:  35
year:  2014 , day:  36
year:  2014 , day:  37
year:  2014 , day:  38
year:  2014 , day:  39
year:  2014 , day:  40
year:  2014 , day:  41
year:  2014 , day:  42
year:  2014 , day:  43
year:  2014 , day:  

In [27]:
if export_csv == True:
    np.savetxt(outputpath+'LIMphy_'+str(first_year)+'_'+str(last_year)+'_timeseries.csv', limphy1, delimiter=";")
    np.savetxt(outputpath+'LIMdia_'+str(first_year)+'_'+str(last_year)+'_timeseries.csv', limdia1, delimiter=";")
    print 'exporting done ...'

exporting done ...


In [23]:
# limphy          = np.zeros((len(years),len(mesh.x2)))
# limdia          = np.zeros((len(years),len(mesh.x2)))

# for mo in range(0,11):
#     print mo
#     if mo ==0: 
#         dayind = (range(0,15))
#         month='JAN'
#     if mo ==1: 
#         dayind = (range(15,30))
#         month='FEV'
#     if mo ==2: 
#         dayind = (range(30,45))
#         month='MAR'
#     if mo ==3: 
#         dayind = (range(45,60))
#         month='APR'
#     elif mo ==4: 
#         dayind = (range(60,75))
#         month='MAY'
#     elif mo ==5: 
#         dayind = (range(75,90))
#         month='JUN'
#     elif mo ==6: 
#         dayind = (range(90,105))
#         month='JUL'
#     elif mo ==7:
#         dayind = (range(105,120))
#         month='AUG'
#     elif mo ==8: 
#         dayind = (range(120,135))
#         month='SEP'
#     elif mo ==9: 
#         dayind = (range(135,150))
#         month='OCT'
#     elif mo ==10: 
#         dayind = (range(150,165))
#         month='NOV'
#     elif mo ==11: 
#         dayind = (range(165,174))
#         month='DEC'
        
#     for ye in range(0,len(years)):
#      print years[ye]
    
#      ncfile  = resultpath+runid+'.'+str(years[ye])+'.oce.mean.nc'
#      f0       = Dataset(ncfile, 'r')  
    
#      ncfile  = resultpath+runid+'.'+str(years[ye])+'.bio.mean.nc'
#      f1       = Dataset(ncfile, 'r')   
        
#      if years[ye] < 2000:
#         DIN       = f0.variables[din_id][mo,0:mesh.n2d]
#         DSi       = f0.variables[dsi_id][mo,0:mesh.n2d]
#         DFe       = f0.variables[dfe_id][mo,0:mesh.n2d]
#         phyc      = f0.variables[phyc_id][mo,0:mesh.n2d]
#         phyn      = f0.variables[phyn_id][mo,0:mesh.n2d]
#         phychl    = f0.variables[phychl_id][mo,0:mesh.n2d]
#         diac      = f0.variables[diac_id][mo,0:mesh.n2d]
#         dian      = f0.variables[dian_id][mo,0:mesh.n2d]
#         diasi     = f0.variables[diasi_id][mo,0:mesh.n2d]
#         diachl    = f0.variables[diachl_id][mo,0:mesh.n2d]
#         T         = f0.variables[temp_id][mo,0:mesh.n2d]
#         par         = f1.variables[par_id][mo,0:mesh.n2d]
      
#      else:
        
#         DIN       = f0.variables[din_id][dayind,0:mesh.n2d].mean(axis=0)
#         DSi       = f0.variables[dsi_id][dayind,0:mesh.n2d].mean(axis=0)
#         DFe       = f0.variables[dfe_id][dayind,0:mesh.n2d].mean(axis=0)
#         phyc      = f0.variables[phyc_id][dayind,0:mesh.n2d].mean(axis=0)
#         phyn      = f0.variables[phyn_id][dayind,0:mesh.n2d].mean(axis=0)
#         phychl    = f0.variables[phychl_id][dayind,0:mesh.n2d].mean(axis=0)
#         diac      = f0.variables[diac_id][dayind,0:mesh.n2d].mean(axis=0)
#         dian      = f0.variables[dian_id][dayind,0:mesh.n2d].mean(axis=0)
#         diasi     = f0.variables[diasi_id][dayind,0:mesh.n2d].mean(axis=0)
#         diachl    = f0.variables[diachl_id][dayind,0:mesh.n2d].mean(axis=0)
#         T         = f0.variables[temp_id][dayind,0:mesh.n2d].mean(axis=0)
#         par       = f1.variables[par_id][dayind,0:mesh.n2d].mean(axis=0)

#      Nlimphy = DIN/(DIN+0.55)
#      Nlimdia = DIN/(DIN+1.0)
#      Silim   = DSi/(DSi+4.0)
#      Felimphy= DFe/(DFe+0.02)
#      Felimdia= DFe/(DFe+0.12)
    
#      # Quotas
#      phychlQ = phychl/phyc
#      phynQ   = phyn/phyc
#      diachlQ = diachl/diac
#      dianQ   = dian/diac
#      diasiQ  = diasi/diac
     
#      # Temperature dependance
#      T = 1./(T + C2K)
#      T = np.exp( -Ae * (T - rTref));
    
#      # Nutrient growth limitation
#      Phy_dq              = NCmin - phyn
#      Phy_recom_limiter   = 1.0 - np.exp( -NMinSlope*( abs(Phy_dq)-Phy_dq )**2)

#      Dia_dq              = NCmin - dian
#      Dia_recom_limiterN  = 1.0 - np.exp( -NMinSlope*( abs(Dia_dq)-Dia_dq )**2)
#      Dia_dq              = SiCmin - diasiQ;
#      Dia_recom_limiterSi = 1.0 - np.exp( -SiMinSlope*( abs(Dia_dq)-Dia_dq )**2)
        
#      # Most limiting factor 
#      Fephy       = DFe/(DFe+0.02)
#      Phy_qlimFac = np.column_stack((Phy_recom_limiter,Fephy))
#      Phy_qlimFac = Phy_qlimFac.min(axis=1)
        
#      Fedia       = DFe/(DFe+0.12)
#      Dia_qlimFac = np.column_stack((Dia_recom_limiterN,Dia_recom_limiterSi,Fedia))
#      Dia_qlimFac = Dia_qlimFac.min(axis=1)
    
#      # pmax
#      Pmax_phy = Pcm * Phy_qlimFac * T
#      Pmax_dia = Pcm * Dia_qlimFac * T

#      Phy_Light_limiter = 1 - np.exp((-alfa_phy * phychlQ * par)/Pmax_phy)
#      Dia_Light_limiter = 1 - np.exp((-alfa_dia * diachlQ * par)/Pmax_dia)
    
#      # Most limiting factors found for nano
#      lim=np.column_stack((Felimphy,Nlimphy,Phy_Light_limiter))
#      limphy[ye,:]=lim.argmin(axis=1)
#      # Most limiting factors found for dia
#      lim=np.column_stack((Felimdia,Nlimdia,Silim,Dia_Light_limiter))
#      limdia[ye,:]=lim.argmin(axis=1)

    
#     if export_csv == True:
#         np.savetxt(outputpath+'LIMphy_'+month+'_'+str(first_year)+'_'+str(last_year)+'_timeseries.csv', limphy, delimiter=";")
#         np.savetxt(outputpath+'LIMdia_'+month+'_'+str(first_year)+'_'+str(last_year)+'_timeseries.csv', limdia, delimiter=";")
#         print 'exporting done ...'

0
1980


/home/hbkoziel/.conda/envs/pyfesom-docs/lib/python2.7/site-packages/ipykernel_launcher.py:120: RuntimeWarning: divide by zero encountered in divide
/home/hbkoziel/.conda/envs/pyfesom-docs/lib/python2.7/site-packages/ipykernel_launcher.py:121: RuntimeWarning: divide by zero encountered in divide


1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
exporting done ...
1
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
exporting done ...
2
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
exporting done ...
3
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
exporting done ...
4
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
exporting done ...
5
